In [2]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import os
from zipfile import ZipFile
from os import listdir
from os.path import isfile, join
import datetime
import time
import os
import numpy as np

In [39]:
zf = ZipFile(f'data/firstratedata/crypto-active_30min_wnk5g.zip') 
data_files = zf.filelist
data = []
for file in data_files:
    if file.filename.startswith('USDC') or file.filename.startswith('USDT') or file.filename.startswith('BTC-EUR') or file.filename.startswith('ETH-BTC') or file.filename.startswith('UST'):
        continue
    print(file.filename)
    df = pd.read_csv(zf.open(file), names=['datetime', 'open', 'high', 'low', 'close', 'volume'])
#     df['datetime'] = df['datetime'].apply(lambda x: f'{x[:4]}-{x[4:6]}-{x[6:8]}T{x[9:11]}:{x[11:13]}:{x[13:15]}')
    df['day'] = df['datetime'].apply(lambda x: x.split(' ')[0])
    df['month'] = df['datetime'].apply(lambda x: x.split(' ')[0][:7])
#     df['minutes'] = df['datetime'].apply(lambda x: int(x.split(' ')[1][:2])*60+int(x.split(' ')[1][3:5]))
    print(len(df), len(data))

    df['file'] = file.filename
    data.append(df)
    

ADA_30min.txt
56256 0
BAT_30min.txt
48630 1
BCH_30min.txt
63290 2
BNT_30min.txt
64080 3
BSV_30min.txt
45956 4
BTC_30min.txt
144094 5
BTG_30min.txt
63907 6
CVC_30min.txt
20675 7
DAI_30min.txt
41232 8
DASH_30min.txt
73640 9
DCR_30min.txt
10652 10
DOGE_30min.txt
71890 11
EOS_30min.txt
70846 12
ETC_30min.txt
79262 13
ETH_30min.txt
92033 14
FUN_30min.txt
33019 15
HT_30min.txt
10536 16
ICX_30min.txt
46549 17
IOST_30min.txt
42233 18
KNC_30min.txt
26541 19
LINK_30min.txt
35802 20
LRC_30min.txt
17991 21
LSK_30min.txt
49755 22
LTC_30min.txt
79383 23
MAID_30min.txt
69784 24
MANA_30min.txt
37648 25
MKR_30min.txt
35369 26
NEO_30min.txt
67534 27
OMG_30min.txt
69245 28
ONT_30min.txt
45444 29
PAX_30min.txt
15170 30
QTUM_30min.txt
49346 31
REP_30min.txt
61160 32
SC_30min.txt
20197 33
SNT_30min.txt
27065 34
TRX_30min.txt
64797 35
UTK_30min.txt
26648 36
VET_30min.txt
33642 37
WAVES_30min.txt
23813 38
XEM_30min.txt
69957 39
XLM_30min.txt
59066 40
XMR_30min.txt
78566 41
XRP_30min.txt
77009 42
XTZ_30min.txt

In [1]:
data[0].info()

NameError: name 'data' is not defined

In [41]:
data[0].iloc[-50:]

,datetime,open,high,low,close,volume,day,month,file
56206,2021-07-15 23:00:00,1.229,1.238,1.226,1.235,487199.559,2021-07-15,2021-07,ADA_30min.txt
56207,2021-07-15 23:30:00,1.236,1.236,1.217,1.224,1158247.860,2021-07-15,2021-07,ADA_30min.txt
56208,2021-07-16 00:00:00,1.225,1.233,1.217,1.223,780896.298,2021-07-16,2021-07,ADA_30min.txt
56209,2021-07-16 00:30:00,1.223,1.225,1.210,1.221,998538.188,2021-07-16,2021-07,ADA_30min.txt
56210,2021-07-16 01:00:00,1.221,1.234,1.221,1.232,475890.635,2021-07-16,2021-07,ADA_30min.txt
56211,2021-07-16 01:30:00,1.232,1.237,1.231,1.235,412062.291,2021-07-16,2021-07,ADA_30min.txt
56212,2021-07-16 02:00:00,1.236,1.245,1.235,1.239,690915.853,2021-07-16,2021-07,ADA_30min.txt
56213,2021-07-16 02:30:00,1.239,1.240,1.234,1.238,496045.368,2021-07-16,2021-07,ADA_30min.txt
56214,2021-07-16 03:00:00,1.238,1.240,1.235,1.239,371414.441,2021-07-16,2021-07,ADA_30min.txt
56215,2021-07-16 03:30:00,1.240,1.243,1.236,1.237,297609.681,2021-07-16,2021-07,ADA_30min.txt


In [6]:
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')

In [7]:
seq_len = 2048

In [69]:
tokenizer.decode([50256])

'<|endoftext|>'

In [43]:
market = 'crypto'
frame = '30min'
samples_tokenized = []
for i in range(len(data)):
    symbol = data[i]['file'].iloc[0].split('_')[0] + 'USD'
    print(symbol)
    pbar = tqdm(enumerate(data[i].groupby('month')), total=len(data[i]['month'].unique()))
    for idx, (day, group) in pbar:
        group = group.round(6)
        prompt_txt = f"{market}_{symbol}_{day}\n;"
        prompt = tokenizer.encode(prompt_txt)
        t = [f"{row['datetime'].split(' ')[1][:5]},{row['open']},{row['high']},{row['low']},{row['close']},{int(row['volume'])}" for idx, row in group.iterrows()]
        tokens = tokenizer.encode(';'.join(t))
        
        token_len = (seq_len - len(prompt))
        iters = len(tokens) // token_len
        
        sample = prompt + tokens[:token_len]
        while len(sample) == seq_len:  
            samples_tokenized.append(sample)
            pbar.set_postfix(size=len(samples_tokenized))
                
            j = 0
            tokens = tokens[token_len:]
            while j < len(tokens) - token_len and tokens[j] != 26:
                j += 1
            tokens = tokens[j+1:]
            sample = prompt + tokens[:token_len]

  0%|                                                                                                                                                                                                      | 0/41 [00:00<?, ?it/s, size=1]

ADAUSD


  2%|████▎                                                                                                                                                                                       | 1/43 [00:00<00:07,  5.43it/s, size=675]

BATUSD


  0%|                                                                                                                                                                                                              | 0/44 [00:00<?, ?it/s]

...

  5%|█████████▊                                                                                                                                                                                | 2/38 [00:00<00:01, 19.42it/s, size=27139]

ZILUSD


  0%|                                                                                                                                                                                                              | 0/47 [00:00<?, ?it/s]

ZRXUSD


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 47/47 [00:11<00:00,  4.15it/s, size=28389]


In [65]:
tokenizer.decode(samples_tokenized[2])

'crypto_BTCUSD_2021-06-25\n;02:36,35273.0,35336.11,35252.9,35295.94,95;02:37,35294.4,35294.4,35248.88,35254.71,13;02:38,35254.71,35314.60494,35254.26723,35290.02,12;02:39,35299.69,35385.92,35272.36,35341.8,41;02:40,35339.01,35375.5,35314.92,35328.77,17;02:41,35334.10885,35355.0,35279.19,35279.19,18;02:42,35288.0,35330.0,35256.72,35261.36,13;02:43,35266.58311,35311.02,35245.67,35311.02,35;02:44,35305.0,35314.05,35247.72,35260.38,7;02:45,35255.0,35279.45,35229.88,35235.83,9;02:46,35247.57,35251.0,35163.4,35163.4,28;02:47,35163.37,35215.92,35158.35361,35180.56,23;02:48,35158.5,35180.0,35097.91,35109.78,20;02:49,35109.78,35158.95,35108.11,35152.96,12;02:50,35152.96,35152.96,35122.0,35134.9,5;02:51,35135.05,35145.59,35077.54,35117.77,10;02:52,35117.76,35153.0,35108.2,35111.0,14;02:53,35111.0,35112.59,35056.72,35076.01,27;02:54,35076.01,35113.33,35020.66,35113.33,33;02:55,35077.58,35170.0,35077.58,35130.04,27;02:56,35130.05,35183.91,35126.9,35150.07,10;02:57,35150.08,35182.44,35144.2,35163.1

In [44]:
samples_tokenized = np.array(samples_tokenized)

In [57]:
samples_tokenized = np.concatenate((np.load('all_crypto_samples.npy'), np.load('all_crypto_samples_5min.npy'), np.load('all_crypto_samples_30min.npy'), np.load('all_crypto_samples_1hour.npy')))

In [58]:
samples_tokenized.shape

(846263, 2048)

In [68]:
846263/46263

18.292436720489377

In [46]:
np.save(f'all_crypto_samples_{frame}', samples_tokenized)

In [59]:
def _int64_feature(value):
    """
    Returns an int64_list from a bool / enum / int / uint.
    """
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

In [60]:
def write_to_file(writer, data):
    """
    writes data to tfrecord file
    """
    feature = {
        "text": _int64_feature(data)
    }
    tf_example = tf.train.Example(features=tf.train.Features(feature=feature))
    writer.write(tf_example.SerializeToString())

In [61]:
def split_list(l, n):
    # splits list/string into n size chunks
    return [l[i:i + n] for i in range(0, len(l), n)]

In [62]:
def write_files(files, files_per, output_dir, out_name, start_no, write_remainder=True, process_no=None):
    # writes a list of files to .tfrecords
    chunks = split_list(files, files_per)

    if len(chunks[-1]) != files_per and not write_remainder:  # pop the last file if it's length != files per
        remainder = chunks.pop(-1)
    else:
        remainder = None  # assuming files = remainder from an old chunk here
        files_per = len(chunks[-1])

    for files in chunks:
        fp = f"{output_dir}/{out_name}_{start_no}"
        if process_no is not None:
            fp += f"_{process_no}"
        fp += f"_{files_per}"  # add number of files in tfrecord to end of fp
        fp += ".tfrecords"
        with tf.io.TFRecordWriter(fp) as writer:
            for f in files:
                write_to_file(writer, f)
        start_no += 1
    return start_no, remainder

In [63]:
def create_tfrecords(tokenized_files_array):
    files_per = 100000
    _tfrecord_count, remainder = write_files(tokenized_files_array, files_per=files_per,
                                                         output_dir='records', out_name=f'crypto',
                                                         start_no=0, process_no=None)

In [64]:
import tensorflow as tf

In [66]:
results = create_tfrecords(samples_tokenized)

In [23]:
@tf.function
def decode_fn(record_bytes):
  return tf.io.parse_single_example(
      # Data
      record_bytes,

      # Schema
      {"x": tf.io.FixedLenFeature([], dtype=tf.float32),
       "y": tf.io.FixedLenFeature([], dtype=tf.float32)}
  )

In [ ]:
for batch in tf.data.TFRecordDataset(['records/crypto_1min_0_73174.tfrecords']).map(decode_fn):
    print("x = {x:.4f},  y = {y:.4f}".format(**batch))

In [19]:
sum([len(samples_tokenized[i])==1024 for i in range(len(samples_tokenized))])/len(samples_tokenized)

1.0

In [21]:
max(max(samples_tokenized))

49541

In [4]:
samples_tokenized = np.load('all_crypto_samples.npy', allow_pickle=True)

### Prepare the dataset

In [5]:
class SongLyrics(Dataset):
    
    def __init__(self, data, gpt2_type="gpt2", max_length=1024):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.lyrics = data
        
    def __len__(self):
        return len(self.lyrics)

    def __getitem__(self, item):
        return torch.tensor(self.lyrics[item]).long()

In [6]:
dataset = SongLyrics(samples_tokenized, gpt2_type="gpt2", max_length=seq_len)

In [7]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp
get_n_params(model), np.log(5), np.log(dataset.__len__()/get_n_params(model)), dataset.__len__()/get_n_params(model)

(124439808, 1.6094379124341003, -5.489264652483911, 0.0041308806905262984)

##### Prepare training

In [11]:
del model

In [100]:
torch.cuda.empty_cache()

In [8]:
def train(
    dataset, model, tokenizer, batch_size_loader=32, epochs=1, 
    lr=2.5e-5, warmup_steps=200,
    gpt2_type="gpt2", output_dir=".", output_prefix="attempt1",
    test_mode=False,save_model_on_epoch=True,
):
    try:
        device=torch.device("cuda")
        model = model.cuda()
        model.train()

        optimizer = AdamW(model.parameters(), lr=lr)
        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
        )

        batch_size = 512//batch_size_loader
        train_dataloader = DataLoader(dataset, batch_size=batch_size_loader, shuffle=True)
        loss=0
        accumulating_batch_count = 0
        input_tensor = None
        
        acc = []
        acc_sign = []
        mae = []

        for epoch in range(epochs):

            print(f"Training epoch {epoch}")
            print(loss)
            pbar = tqdm(enumerate(train_dataloader), total=train_dataloader.__len__())
            for idx, input_tensor in pbar:
                input_tensor = input_tensor.to(device)
                outputs = model(input_tensor, labels=input_tensor)
                loss = outputs[0]
                loss.backward()
                
#                 inp = input_tensor.cpu().detach().numpy()
#                 logits = outputs['logits'].argmax(axis=2).cpu().detach().numpy()

#                 acc.append((logits == inp).sum()/logits.size)
#                 acc_sign.append((np.sign(logits-len(clusters)//2) == np.sign(inp-len(clusters)//2)).sum()/logits.size)
#                 mae.append(mean_absolute_error(inp, logits)/len(clusters))

                if (accumulating_batch_count % batch_size) == 0:
                    optimizer.step()
                    scheduler.step()
                    optimizer.zero_grad()
                    model.zero_grad()

                accumulating_batch_count += 1
                input_tensor = None
#                 pbar.set_postfix(loss=loss.cpu().detach().numpy(), ACC=acc[-1], ACC_sign=acc_sign[-1], a_mae=mae[-1],
#                                  lr=scheduler.get_last_lr()[0])
            if save_model_on_epoch:
                torch.save(
                    model.state_dict(),
                    os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
                )    
    except Exception as e:
        print(f'{str(e)}')
        try:
            del model
            del optimizer
            del scheduler
            del train_dataloader
            del loss
            del input_tensor
            del outputs
        except:
            pass
        
        torch.cuda.empty_cache()
        model = GPT2LMHeadModel.from_pretrained('gpt2')
    return model

### Actual Training

In [105]:
torch.cuda.empty_cache()

In [ ]:
#Train the model on the specific data we have
model = train(dataset, model, tokenizer, batch_size_loader=4)

  0%|                                                                                                                                                                                                          | 0/128512 [00:00<?, ?it/s]

Training epoch 0
0


 11%|████████████████████▋                                                                                                                                                                    | 14368/128512 [1:33:01<13:00:51,  2.44it/s]

In [ ]:
a

In [ ]:
B

In [ ]:
#Rouge score
from rouge import Rouge
rouge=Rouge()

rouge.get_scores(test_set['Generated_lyrics'], test_set['True_end_lyrics'], avg=True, ignore_empty=True)

In [ ]:
!pip install rouge

In [ ]:
#Using BLEU score to compare the real sentences with the generated ones
import statistics
from nltk.translate.bleu_score import sentence_bleu

scores=[]

for i in range(len(test_set)):
  reference = test_set['True_end_lyrics'][i]
  candidate = test_set['Generated_lyrics'][i]
  scores.append(sentence_bleu(reference, candidate))

statistics.mean(scores)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.4075527115657135

In [ ]:
#Finish the sentences when there is a point, remove after that
final=[]

for i in range(len(test_set)):
  to_remove = test_set['Generated_lyrics'][i].split('.')[-1]
  final.append(test_set['Generated_lyrics'][i].replace(to_remove,''))

test_set['Generated_lyrics'] = final
test_set.head()

,level_0,index,SName,Lyric,Artist,Genre,True_end_lyrics,Generated_lyrics
0,2946,3317,Do the Clam,(Words & music by Wayne - Weisman - Fuller). H...,Elvis Presley,Rock,Grab your barefoot baby by the hand. Turn and ...,
1,12130,13349,Elevation,"High, higher than the sun. You shoot me from a...",U2,Rock,in the sky. You make me feel like I can fly. S...,on earth.\nI start reading monographs about J...
2,596,640,Professional Torturer,Infatuation. Court well meant. 'Cause I'm the ...,Alanis Morissette,Rock,I renounce my name. Professional torturer. I d...,
3,3733,4116,I Am Yours,I am yours. However distant you may be. There ...,Eric Clapton,Rock,me. Each memory that has left its trace with m...,
4,11961,13175,Bombs Away,The general scratches his belly and thinks. Hi...,The Police,Rock,hard and sweet. A military man would love to m...,straight red hair.


In [ ]:
#Loop to keep only generated text and add it as a new column in the dataframe
my_generations=[]

for i in range(len(generated_lyrics)):
  a = test_set['Lyric'][i].split()[-30:] #Get the matching string we want (30 words)
  b = ' '.join(a)
  c = ' '.join(generated_lyrics[i]) #Get all that comes after the matching string
  my_generations.append(c.split(b)[-1])

test_set['Generated_lyrics'] = my_generations

In [ ]:
#Function to generate multiple sentences. Test data should be a dataframe
def text_generation(test_data):
  generated_lyrics = []
  for i in range(len(test_data)):
    x = gen_text(test_data['Lyric'][i], tokenizer, model)
    generated_lyrics.append(x)
  return generated_lyrics

generated_lyrics = text_generation(test_set)

In [ ]:
#Generate sequences
gen_text(df['Lyric'][0],tokenizer,model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['I feel so unsure. As I take your hand and lead to the dance floor. As the music dies, something in your eyes. Calls to mind the silver screen. And all its sad good-byes. I\'m never gonna dance again. Guilty feet have got no rhythm. Though it\'s easy to pretend. I know you are not a fool. Should\'ve known better than to cheat a friend. And waste the chance that I\'ve been given. So I\'m never gonna dance again. The way I danced with you. Time can never mend. The careless whispers of a good friend. To the heart and mind. Ignorance is kind. There\'s no comfort in the truth. Pain is all you\'ll find. I\'m never gonna dance again. Guilty feet have got no rhythm. Though it\'s easy to pretend. I know you are not a fool. Should\'ve known better than to cheat a friend. And waste this chance that I\'ve been given. So I\'m never gonna dance again. The way I danced with you. Never without your love. Tonight the music seems so loud. I wish that we could lose this crowd. Maybe it\'s better this wa

In [ ]:
## Making a function that will generate text for us ##
def gen_text(prompt_text, tokenizer, model, n_seqs=1, max_length=374):
  # n_seqs is the number of sequences to generate
  # max_length is the maximum length of the sequence
  encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")
  # We are encoding the text using the gpt tokenizer. The return tensors are of type "pt"
  # since we are using PyTorch, not tensorflow
  output_sequences = model.generate(
      input_ids=encoded_prompt,
      max_length=max_length+len(encoded_prompt), # The model has to generate something, 
      # so we add the length of the original sequence to max_length
      temperature=1.0,
      top_k=0,
      top_p=0.9,
      repetition_penalty=1.2, # To ensure that we dont get repeated phrases
      do_sample=True,
      num_return_sequences=n_seqs
  ) # We feed the encoded input into the model.
  ## Getting the output ##
  if len(output_sequences.shape) > 2:
    output_sequences.squeeze_() # the _ indicates that the operation will be done in-place
  generated_sequences = []
  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = tokenizer.decode(generated_sequence)
    total_sequence = (
        prompt_text + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True, )) :]
    )
    generated_sequences.append(total_sequence)
  return generated_sequences

In [ ]:
tokenizer = transformers.GPT2Tokenizer.from_pretrained('gpt2')
model = transformers.GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
import transformers
import torch

# GPT2 without any fine Tuning

In [ ]:
#Rouge score
from rouge import Rouge
rouge=Rouge()

rouge.get_scores(test_set['Generated_lyrics'], test_set['True_end_lyrics'], avg=True)

{'rouge-1': {'f': 0.33620873608456614,
  'p': 0.3805105543072668,
  'r': 0.33900000000000013},
 'rouge-2': {'f': 0.24573902727265526,
  'p': 0.280178576490597,
  'r': 0.252700228832952},
 'rouge-l': {'f': 0.3756182538370741,
  'p': 0.40754447860807824,
  'r': 0.39803790370276443}}

In [ ]:
#Using BLEU score to compare the real sentences with the generated ones
import statistics
from nltk.translate.bleu_score import sentence_bleu

scores=[]

for i in range(len(test_set)):
  reference = test_set['True_end_lyrics'][i]
  candidate = test_set['Generated_lyrics'][i]
  scores.append(sentence_bleu(reference, candidate))

statistics.mean(scores)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.6848624352005677

### Analyze performance

In [ ]:
test_set['True_end_lyrics'][7]

"the. Woman without pride x 5. You don't see things like I do. You don't see things. Like I do."

In [ ]:
test_set['Generated_lyrics'][7]

" in that. Yes we've heard the great thing. I know what you've heard. You told me we've been promised so much."

In [ ]:
#Finish the sentences when there is a point, remove after that
final=[]

for i in range(len(test_set)):
  to_remove = test_set['Generated_lyrics'][i].split('.')[-1]
  final.append(test_set['Generated_lyrics'][i].replace(to_remove,''))

test_set['Generated_lyrics'] = final
test_set.head()

,index,SName,Lyric,Artist,Genre,True_end_lyrics,Generated_lyrics
0,2834,Don't Bring Me Down,"Hm hm hm. Hm hm hm. I'm on my own, nowhere to ...",David Bowie,Rock,me down. Until then I'll settle down. Say I'll...,me down. I want to see your face. I want to s...
1,6725,Flight,I've lost my balance. I fell from the trapeze....,Lifehouse,Rock,more falling). (No more striving). Only flying...,more falling). No more striving. No more hear...
2,7233,Black And Gold,If the fish swam out of the ocean. and grew le...,Lulu Santos,Rock,matter. 'cause if you're not really here. then...,magic words. 'cause if you're not real here. ...
3,2525,Hear My Train A Comin,"""YEAH, I SEE WE GOT A FEW FRIENDS LAYIN' ROUND...",Cássia Eller,Rock,YOU VERY MUCH THANK YOU VERY MUCH. THANX A LOT...,"S TO YOU. YES, YES, YES. YES. YES, YES."
4,1778,The Hills Of Mexico,'Twas in the town of griffin. In the year of s...,Bob Dylan,Rock,the cattle run. I tread towards the hiding pla...,"a horrid start. Struck my wagon, but couldn't..."


In [ ]:
#Loop to keep only generated text and add it as a new column in the dataframe
my_generations=[]

for i in range(len(generated_lyrics)):
  a = test_set['Lyric'][i].split()[-30:] #Get the matching string we want (30 words)
  b = ' '.join(a)
  c = ' '.join(generated_lyrics[i]) #Get all that comes after the matching string
  my_generations.append(c.split(b)[-1])

test_set['Generated_lyrics'] = my_generations

In [ ]:
generated_lyrics = text_generation(test_set)

In [ ]:
#Function to generate multiple sentences. Test data should be a dataframe
def text_generation(test_data):
  generated_lyrics = []
  for i in range(len(test_data)):
    x = generate(model.to('cpu'), tokenizer, test_data['Lyric'][i], entry_count=1)
    generated_lyrics.append(x)
  return generated_lyrics

In [ ]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=30, #maximum number of words
    top_p=0.8,
    temperature=1.,
):

    model.eval()

    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False

            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break
            
            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
              generated_list.append(output_text)
                
    return generated_list

In [ ]:
#Load the model to use it
model = torch.load('/content/drive/MyDrive/Google colabs/model.pt')

###### Text generation